# Evaluation on the test(validation) dataset

In [ ]:
from utils.miscellaneous import read_config
from sklearn.metrics import confusion_matrix

import os
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import utils.train_val_test_dataset_import as tvt
import utils.evaluation_matrix as em


##  1 Parse configuration file + initializations

In [ ]:
# read config files
cfg = read_config('./config.yaml')

# constants
num_classes = cfg['num_classes']
image_height = cfg['image_height']
image_width = cfg['image_width']
batch_size = cfg['batch_size']['tra']

labels = cfg['labels']

# paths
path_test = cfg['Path']['path_test']
save_misclassified_images_path = cfg['Path']['save_misclassified_images_path']
# model_path = cfg['Path']['model_path']

# model_path = r"F:\Tianlong\PythonProject\deep_plastic\output\TL_vs_scratch\model_trained_for_GradCam\weights\mobile_net_lr_0.0001_TL_all_GC.hdf5"
model_path = r"F:\Tianlong\PythonProject\deep_plastic\output\TL_vs_scratch\from_scratch_weights\mobile_net_lr_0.0001.hdf5"

## 2 Load test dataset

In [ ]:
ds_test = tvt.import_dataset_test(
    path_test, image_height, image_width, batch_size)

# autotune
AUTOTUNE = tf.data.AUTOTUNE
ds_test = ds_test.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
ds_test

In [ ]:
# Visualizing the test DATA

plt.figure(figsize=(10, 10))
for images, label_s in ds_test.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1) 
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.axis("off")

## 3 Load model

In [ ]:
model = tf.keras.models.load_model(model_path)
model.summary()

## 3 Test

### 3.1 Statistics output

In [ ]:
# statistics output when evaluated on the test data
 
loss, acc = model.evaluate(ds_test)
print("Model, accuracy: {:5.2f}%".format(100 * acc))

In [ ]:
em.cnn_statistics(model,ds_test)

### 3.2 Confusion matrix 

In [ ]:
em.ConfusionMatrix('MobileNet', model, ds_test, num_classes)


### 3.3 Show and save Misclassified examples

In [ ]:
y_true = np.concatenate([y for x, y in ds_test], axis=0)
y_pred = model.predict(ds_test).argmax(axis=1)
x_test = np.concatenate([x for x, y in ds_test], axis=0)
x_test = x_test/255.0
misclassified_idx = np.where(y_pred != y_true)[0]
print("Number of misclassified images: ", misclassified_idx.size)


In [ ]:
# misclassified_idx

In [ ]:
# Save misclassified images in one folder

for i in misclassified_idx:
    fig = plt.figure(figsize=(2.24,2.24)) 
    plt.axis('off')
    plt.grid(False)
    plt.imshow(x_test[i], cmap='gray')
    plt.title("%s pred as: %s" % (labels[y_true[i]], labels[y_pred[i]]))
    fig_name = 'misclass' + str(i) + ".jpg"
    fig_path = os.path.join(save_misclassified_images_path,fig_name)
    fig.savefig(fig_path)
    plt.close() # close the figure, to prevent the figure shows up in Notebook

In [ ]:
# Show one random misclassified example
i = np.random.choice(misclassified_idx)
fig = plt.figure(figsize=(10,8))
plt.grid(False)
plt.imshow(x_test[i], cmap='gray')
plt.title("True label: %s, Predicted: %s" % (labels[y_true[i]], labels[y_pred[i]]));

In [ ]:
# Show one specific misclassified example
fig = plt.figure(figsize=(10,8))
plt.grid(False)
plt.imshow(x_test[0], cmap='gray')
plt.title("True label: %s, Predicted: %s" % (labels[y_true[160]], labels[y_pred[0]]));

# Majority Vote

In [ ]:
num_classes=4
y_true = np.concatenate([y for x, y in ds_val], axis=0)
y_pred_squeeze = model_squeeze.predict(ds_val).argmax(axis=1)
y_pred_dense = model_dense.predict(ds_val).argmax(axis=1)
y_pred_incept = model_incept.predict(ds_val).argmax(axis=1)
major = [0]*len(y_pred_incept)

In [ ]:
data = {'squeezenet': y_pred_squeeze, 'densenet121': y_pred_dense, 'inceptionv3': y_pred_incept, 'm_vote': major }

In [ ]:
df = pd.DataFrame(data)

In [ ]:
for i in range(0, len(data['squeezenet'])):
    if (data['squeezenet'][i] != data['densenet121'][i]) and (data['squeezenet'][i] != data['inceptionv3'][i]) and (data['densenet121'][i] != data['inceptionv3'][i]):
        df['m_vote'][i] = data['squeezenet'][i]
    elif (data['squeezenet'][i] == data['densenet121'][i]):
        df['m_vote'][i] = data['squeezenet'][i]
    elif (data['squeezenet'][i] == data['inceptionv3'][i]):
        df['m_vote'][i] = data['squeezenet'][i]
    elif (data['densenet121'][i] == data['inceptionv3'][i]):
        df['m_vote'][i] = data['densenet121'][i]
    else:
        df['m_vote'][i] = data['squeezenet'][i]

In [ ]:
y_pred = np.array(df['m_vote'])

In [ ]:
model = tf.keras.models.load_model(r'C:\Users\Andre\Desktop\Research paper\from_scratch_weights_with_DA\dense_net_DAHV.hdf5')

In [ ]:
y_pred = model.predict(ds_val).argmax(axis=1)

In [ ]:
print(f'Print classification report on validation(test) dataset')
print(f'-----'*10)
print(classification_report(y_true, y_pred))

  # we quickly create a decent looking CM for many classes using pandas and seaborn
print(f'\nPlot confusion matrix on validation(test) dataset')
print(f'-----'*10)
df_cm = pd.DataFrame(confusion_matrix(y_true,y_pred))
f, ax = plt.subplots(1,figsize = (10,10))
sns.heatmap(df_cm, annot=True, annot_kws={"fontsize":12}, ax= ax, vmax=50, cbar=False, cmap='Blues', fmt='d') # we use this value of vmax to emphasize misclassifications
ax.set_title('SqueezeNet - TRAIN: Train 3, Test: Test 4', fontsize=16, fontweight='bold')
ax.set_ylabel('True class');
ax.set_xlabel('Predicted class');

# Majority vote with weights

In [ ]:
y_pred_squeeze = model_squeeze.predict(ds_val)
y_pred_dense = model_dense.predict(ds_val)
y_pred_incept = model_incept.predict(ds_val)

In [ ]:
for i in range(len(y_pred_dense)):
    for j in range(len(y_pred_dense[i])):
        y_pred_dense[i][j] = (0.83*y_pred_incept[i][j]) + (0.86*y_pred_dense[i][j]) + (0.89*y_pred_squeeze[i][j])

In [ ]:
y_pred = np.argmax(y_pred_dense, axis = 1)

In [ ]:
print(f'Print classification report on validation(test) dataset')
print(f'-----'*10)
print(classification_report(y_val, y_pred))


print(f'\nPlot confusion matrix on validation(test) dataset')
print(f'-----'*10)
df_cm = pd.DataFrame(confusion_matrix(y_val,y_pred))
f, ax = plt.subplots(1,figsize = (10,10))
sns.heatmap(df_cm, annot=True, annot_kws={"fontsize":12}, ax= ax, vmax=50, cbar=False, cmap='Blues', fmt='d') # we use this value of vmax to emphasize misclassifications
ax.set_title('Majority Vote - TRAIN: 2.7m/0 deg, VAL: 2.7m/0 deg', fontsize=16, fontweight='bold')
ax.set_ylabel('True class');
ax.set_xlabel('Predicted class');